In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import cvzone
model=YOLO('yolov8m.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        point = [x, y]
        print(point)

C:\Users\ziadz\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0
persondown={}
tracker=Tracker()
counter1=[]

personup={}
counter2=[]
cy1=194
cy2=220
offset=6

In [18]:

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)
cap=cv2.VideoCapture('vidp/vidp.mp4')

while True:    
    ret,frame = cap.read()
    if not ret:
        break
#    frame = stream.read()
    original_video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data.cpu().numpy()
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
   
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5]) # class id
        
        c=class_list[d]
        if 'person' in c:

            list.append([x1,y1,x2,y2])
       
        
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x1,y1,x2,y2,id=bbox
        cx=int(x1+x2)//2
        cy=int(y1+y2)//2
        cv2.circle(frame,(cx,cy),4,(255,0,255),-1)
        if cy1<(cy+offset) and cy1>(cy-offset):
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
            cvzone.putTextRect(frame,f'{id}',(x1,y1),1,2)
            persondown[id]=(cx,cy)

    cvzone.putTextRect(frame,f'person : {len(persondown)}',(50,60),1,2)
    cv2.line(frame,(3,194),(1018,192),(0,255,0),2)
    cv2.line(frame,(5,220),(1019,220),(0,255,255),2)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 24 persons, 301.0ms
Speed: 5.0ms preprocess, 301.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
[513, 444]

0: 320x640 26 persons, 1 suitcase, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 28 persons, 1 backpack, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 25 persons, 1 handbag, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 28 persons, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 29 persons, 1 handbag, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 27 persons, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 29 persons, 12.0ms
Speed: 1.0ms p

In [4]:
import matplotlib.pyplot as plt


In [11]:
type(results)

list

In [34]:
print(results[0].boxes.__doc__)


    A class for storing and manipulating detection boxes.

    Args:
        boxes (torch.Tensor | numpy.ndarray): A tensor or numpy array containing the detection boxes,
            with shape (num_boxes, 6) or (num_boxes, 7). The last two columns contain confidence and class values.
            If present, the third last column contains track IDs.
        orig_shape (tuple): Original image size, in the format (height, width).

    Attributes:
        xyxy (torch.Tensor | numpy.ndarray): The boxes in xyxy format.
        conf (torch.Tensor | numpy.ndarray): The confidence values of the boxes.
        cls (torch.Tensor | numpy.ndarray): The class values of the boxes.
        id (torch.Tensor | numpy.ndarray): The track IDs of the boxes (if available).
        xywh (torch.Tensor | numpy.ndarray): The boxes in xywh format.
        xyxyn (torch.Tensor | numpy.ndarray): The boxes in xyxy format normalized by original image size.
        xywhn (torch.Tensor | numpy.ndarray): The boxes in x

In [36]:
print(results[0].boxes[0])

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.8511], device='cuda:0')
data: tensor([[ 38.5514, 323.2926,  90.5506, 413.6427,   0.8511,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (500, 1020)
shape: torch.Size([1, 6])
xywh: tensor([[ 64.5510, 368.4677,  51.9992,  90.3500]], device='cuda:0')
xywhn: tensor([[0.0633, 0.7369, 0.0510, 0.1807]], device='cuda:0')
xyxy: tensor([[ 38.5514, 323.2926,  90.5506, 413.6427]], device='cuda:0')
xyxyn: tensor([[0.0378, 0.6466, 0.0888, 0.8273]], device='cuda:0')
